In [1]:
import os
from base_fns import get_local_folder

os.chdir(os.path.dirname(get_local_folder()))
import pandas as pd
import tabulate as tb
from script.eval import evaluate
from hydra import compose, initialize
from omegaconf import OmegaConf
import random
from munch import Munch
import numpy as np
from pandarallel import pandarallel
from pathlib import Path
from transformers import BertTokenizerFast as BertTokenizer, BertModel
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

with initialize(
    version_base=None,
    config_path="../cfg",
):
    cfg = compose(config_name="main")

random.seed(cfg.random.seed)
np.random.seed(cfg.random.seed)

/home/paul/CellTriage/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_df = pd.read_csv('data/litsuggest/test.tsv', sep='\t')

In [3]:
test_silver= pd.read_csv('data/litsuggest/silver_test_pred.tsv', sep='\t')
test_silver['PRED'] = test_silver.apply(lambda row: 1 if row['SCORE'] >= 0.5 else 0, axis=1)
test_silver = test_silver[['PMID', 'SCORE', 'PRED']].merge(test_df, how='left', on='PMID')

In [11]:
datasets={'silver': 'data/litsuggest/silver_test_pred.tsv', 'bronze':'data/litsuggest/bronze_test_pred.tsv'}
results = {}
for dataset, fp in datasets.items():
    stub=f"{dataset}/litsuggest"
    print(f"<<{stub}>>")
    df = pd.read_csv(fp, sep='\t')
    df = df[['PMID', 'SCORE']].merge(test_df, how='left', on='PMID')
    y_pred = pd.array(df['SCORE'])
    y_true = pd.array(df['SOURCE'])
    results[stub] = evaluate(y_true, y_pred)

<<silver/litsuggest>>
<<bronze/litsuggest>>


In [14]:
RESULTS_FP = cfg.evaluation.litsuggest_eval.results_fp

df =pd.DataFrame.from_dict(results, orient='index')
with open(RESULTS_FP, "w") as f:
    json_str = df.to_json(orient='records', lines=True)
    f.write(json_str)
    
df

,precision,recall,F1,P10,P100,P200
silver/litsuggest,0.8378,0.9238,0.8787,1.0,0.99,0.985
bronze/litsuggest,0.6531,0.9536,0.7752,1.0,0.94,0.900


: 